In [235]:
from pprint import pprint
import pandas as pd
import numpy as np
import random
import sys
rng = np.random.default_rng(2022)
RECIPIENT_PRO_REVEAL = 52.9
RECIPIENT_CONTRA_REVEAL = 52
PROPORTIONS = 70.92
MODE_DG_DECISION = 0
df = pd.read_csv('./calcs/polar_2022-06-02.csv')
rr_r_session = 'lh73wwd1'
rr_d_session = '8sa9uh3g'

rdf = df[(df['session.code'] == rr_r_session) & (df['player.payable'] == True)]
ddf = df[(df['session.code'] == rr_d_session) & (df['player.payable'] == True)]


def belief_calc(i):
    if i['belief_column_name'] == 'player.own_dg_belief':
        return abs(i['belief_value'] - i['player.dg_decision'])
    if i['belief_column_name'] == 'player.vl_pro_belief':
        return abs(i['belief_value'] - RECIPIENT_PRO_REVEAL)
    if i['belief_column_name'] == 'player.average_dg_belief':
        return abs(i['belief_value'] - MODE_DG_DECISION)
    if i['belief_column_name'] == 'player.vl_contra_belief':
        return abs(i['belief_value'] - RECIPIENT_CONTRA_REVEAL)
    if i['belief_column_name'] == 'player.proportion':
        return abs(i['belief_value'] - PROPORTIONS)
    #
    return 100


# # Now let's calculate payoffs of dictators
rrd_beliefs = ['player.proportion', 'player.average_dg_belief', 'player.vl_pro_belief', 'player.vl_contra_belief']
selected_columns_d = ['participant.label', 'player.dg_decision', 'player.partner_position'] + rrd_beliefs
ddf = ddf[selected_columns_d]
ddf['rpayoff'] = 100 - ddf['player.dg_decision']
ddf['belief_column_name'] = rng.choice(rrd_beliefs, len(ddf), replace=True)
ddf['belief_value'] = ddf.to_numpy()[np.arange(ddf.shape[0]), ddf.columns.get_indexer(ddf['belief_column_name'])]
ddf['belief_match'] = ddf.apply(belief_calc, 1)

ddf['get_belief_payoff'] = ddf['belief_match'] < 11
ddf['belief_payoff'] = ddf['get_belief_payoff'] * 0.25
ddf['total_payoff'] = round(ddf['belief_payoff'] + ddf['rpayoff'] / 100, 2)

ddf.to_csv('calcs/rr_d_payoffs.csv')
print(ddf.total_payoff.sum())
#

# Let's calculate payoffs of receivers
ddf.sort_values(by='player.partner_position', inplace=True)
ddf['row_num'] = np.arange(len(ddf))
ddf.set_index('row_num', inplace=True)
ddf.sort_index(inplace=True)

selected_columns_d = ['participant.label','player.dg_decision', 'player.partner_position']
ddf = ddf[selected_columns_d]
ddf.rename(columns={'participant.label':'dictator_label'}, inplace=True)
rrr_beliefs = ['player.proportion', 'player.own_dg_belief', 'player.vl_pro_belief', 'player.vl_contra_belief']

selected_columns_r = ['participant.label', 'player.recipient_reveal',
                      'player.opinion_war']  + rrr_beliefs

rdf = rdf[selected_columns_r]
rdf['own_position'] =np.where(rdf['player.recipient_reveal']==0, 'nr',
                   np.where(rdf['player.opinion_war']==0, 'no', 'yes'))
rdf.sort_values(by='own_position', inplace=True)
rdf['row_num'] = np.arange(len(rdf))
rdf.set_index('row_num', inplace=True)
rdf.sort_index(inplace=True)
ddf = ddf[['player.dg_decision', 'dictator_label']]

rdf=rdf.join(ddf)
rdf['rpayoff'] = 50+rdf['player.dg_decision']
rdf['belief_column_name'] = rng.choice(rrr_beliefs, len(rdf), replace=True)
rdf['belief_value'] = rdf.to_numpy()[np.arange(rdf.shape[0]),rdf.columns.get_indexer(rdf.belief_column_name)]
rdf['belief_match']= rdf.apply(belief_calc, 1)
rdf['get_belief_payoff']=  rdf['belief_match']<11
rdf['belief_payoff']=  rdf['get_belief_payoff']*0.25
rdf['total_payoff'] = rdf['belief_payoff'] +rdf['rpayoff']/100
print(rdf.total_payoff.sum())
rdf.to_csv('calcs/rr_r_payoffs.csv')









467.79999999999995
209.2
